<h1 align="center"><font size="5">BASIC MOVIE RECOMMENDER SYSTEM</font></h1>

In this project, I will try to recommend other movies to my girlfriend based on her ratings of her 10 favorites movies. The recommendation engine is very basic. It is purely based on the genres of the movies. This project is inspired from one of the hands-on of the <a href="https://www.coursera.org/professional-certificates/ibm-data-science"> IBM Data Science Professional certificate</a> from Coursera. More precisely, it is similar to the lab entitled Content-based Recommendation Systems of the Machine Learning Course of that track.

<h1>Table of contents</h1>

<div class="alert alert-block alert-info" style="margin-top: 20px">
    <ol>
        <li><a href="#ref1">Acquiring the Data</a></li>
        <li><a href="#ref2">Preprocessing</a></li>
        <li><a href="#ref3">Collaborative Filtering</a></li>
    </ol>
</div>
<br>
<hr>

<a id="ref1"></a>
# Acquiring the Data

In [59]:
import pandas as pd
import numpy as np
#import matplotlib.pyplot as plt
#%matplotlib inline

In [60]:
movies_df=pd.read_csv('movies.csv')
movies_df.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


I will be reusing the dataset that was provided in the hands-on activity. It was not specified where the data were coming from.The data quality is quite good. The dataset contains movies from 1874 to 2016. For each movie is associated to one or several genres.

<a id="ref2"></a>
# Preprocessing

In [61]:
#Using regular expressions to find a year stored between parentheses
#We specify the parantheses so we don't conflict with movies that have years in their titles
movies_df['year'] = movies_df.title.str.extract('(\(\d\d\d\d\))',expand=False)
#Removing the parentheses
movies_df['year'] = movies_df.year.str.extract('(\d\d\d\d)',expand=False)
#Removing the years from the 'title' column
movies_df['title'] = movies_df.title.str.replace('(\(\d\d\d\d\))', '')
#Applying the strip function to get rid of any ending whitespace characters that may have appeared
movies_df['title'] = movies_df['title'].apply(lambda x: x.strip())
movies_df.head()

,movieId,title,genres,year
0,1,Toy Story,Adventure|Animation|Children|Comedy|Fantasy,1995
1,2,Jumanji,Adventure|Children|Fantasy,1995
2,3,Grumpier Old Men,Comedy|Romance,1995
3,4,Waiting to Exhale,Comedy|Drama|Romance,1995
4,5,Father of the Bride Part II,Comedy,1995


With that, let's also split the values in the __Genres__ column into a __list of Genres__ to simplify future use. This can be achieved by applying Python's split string function on the correct column

In [62]:
#Every genre is separated by a | so we simply have to call the split function on |
movies_df['genres'] = movies_df.genres.str.split('|')
movies_df.head()

,movieId,title,genres,year
0,1,Toy Story,"[Adventure, Animation, Children, Comedy, Fantasy]",1995
1,2,Jumanji,"[Adventure, Children, Fantasy]",1995
2,3,Grumpier Old Men,"[Comedy, Romance]",1995
3,4,Waiting to Exhale,"[Comedy, Drama, Romance]",1995
4,5,Father of the Bride Part II,[Comedy],1995


Since keeping genres in a list format isn't optimal for the content-based recommendation system technique, we will use the One Hot Encoding technique to convert the list of genres to a vector where each column corresponds to one possible value of the feature. This encoding is needed for feeding categorical data. In this case, we store every different genre in columns that contain either 1 or 0. 1 shows that a movie has that genre and 0 shows that it doesn't. Let's also store this dataframe in another variable since genres won't be important for our first recommendation system.

In [63]:
#Copying the movie dataframe into a new one since we won't need to use the genre information in our first case.
moviesWithGenres_df = movies_df.copy()

#For every row in the dataframe, iterate through the list of genres and place a 1 into the corresponding column
for index, row in movies_df.iterrows():
    for genre in row['genres']:
        moviesWithGenres_df.at[index, genre] = 1
#Filling in the NaN values with 0 to show that a movie doesn't have that column's genre
moviesWithGenres_df = moviesWithGenres_df.fillna(0)
moviesWithGenres_df.head()

,movieId,title,genres,year,Adventure,Animation,Children,Comedy,Fantasy,Romance,...,Horror,Mystery,Sci-Fi,IMAX,Documentary,War,Musical,Western,Film-Noir,(no genres listed)
0,1,Toy Story,"[Adventure, Animation, Children, Comedy, Fantasy]",1995,1.0,1.0,1.0,1.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2,Jumanji,"[Adventure, Children, Fantasy]",1995,1.0,0.0,1.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,3,Grumpier Old Men,"[Comedy, Romance]",1995,0.0,0.0,0.0,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,4,Waiting to Exhale,"[Comedy, Drama, Romance]",1995,0.0,0.0,0.0,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,5,Father of the Bride Part II,[Comedy],1995,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


<a id="ref3"></a>
# Content-Based recommendation system

Now, let's take a look at how to implement __Content-Based__ or __Item-Item recommendation systems__. This technique attempts to figure out what a user's favourite aspects of an item is, and then recommends items that present those aspects. In our case, we're going to try to figure out the input's favorite genres from the movies and ratings given.

Let's begin by creating an input user to recommend movies to:

Notice: To add more movies, simply increase the amount of elements in the __userInput__. Feel free to add more in! Just be sure to write it in with capital letters and if a movie starts with a "The", like "The Matrix" then write it in like this: 'Matrix, The' .

In [64]:
natInput = [
            {'title':'Avatar', 'rating':4.5},
            {'title':'Darknight Rises, The ', 'rating':4.5},
            {'title':'Pirates of the Caribbean: Dead Man\'s Chest', 'rating':4},
            {'title':"Love Actually", 'rating':3.5},
            {'title':'Little Mermaid, The', 'rating':4.5},
            {'title':'Fast and the Furious, The','rating':2.5},
            {'title':'Inception','rating':2.5},
            {'title':'Sister Act 2: Back in the Habit','rating':4},
            {'title':'Ten Commandments, The':4.5},
            {'title':'Freedom Writers','rating':4}
         ] 
inputMovies = pd.DataFrame(natInput)
inputMovies

,title,rating
0,Iron Man,4.5
1,Black Panther,4.0
2,Ratatouille,4.0
3,Now You See Me,3.5
4,"Fast and the Furious 5, The",4.0
5,Harry Potter and the Deathly Hallows,2.5
6,"Place Beyond the Pines, The",3.0
7,"Imitation Game, The",4.0
8,"I, Robot",5.0
9,"Dark Knight Rises, The",5.0


#### Add movieId to input user
With the input complete, let's extract the input movie's ID's from the movies dataframe and add them into it.

We can achieve this by first filtering out the rows that contain the input movie's title and then merging this subset with the input dataframe. We also drop unnecessary columns for the input to save memory space.

In [66]:
#Filtering out the movies by title
inputId = movies_df[movies_df['title'].isin(inputMovies['title'].tolist())]
inputId

,movieId,title,genres,year
7962,8644,"I, Robot","[Action, Adventure, Sci-Fi, Thriller]",2004
11624,50872,Ratatouille,"[Animation, Children, Drama]",2007
12646,59315,Iron Man,"[Action, Adventure, Sci-Fi]",2008
18355,91529,"Dark Knight Rises, The","[Action, Adventure, Crime, IMAX]",2012
20807,101525,"Place Beyond the Pines, The","[Crime, Drama]",2012
21167,102903,Now You See Me,"[Crime, Mystery, Thriller]",2013


In [67]:
#Then merging it so we can get the movieId. It's implicitly merging it by title.
inputMovies = pd.merge(inputId, inputMovies)
#Dropping information we won't use from the input dataframe
#inputMovies = inputMovies.drop('genres', 1).drop('year', 1)
#Final input dataframe
#If a movie you added in above isn't here, then it might not be in the original 
#dataframe or it might spelled differently, please check capitalisation.
inputMovies

,movieId,title,genres,year,rating
0,8644,"I, Robot","[Action, Adventure, Sci-Fi, Thriller]",2004,5.0
1,50872,Ratatouille,"[Animation, Children, Drama]",2007,4.0
2,59315,Iron Man,"[Action, Adventure, Sci-Fi]",2008,4.5
3,91529,"Dark Knight Rises, The","[Action, Adventure, Crime, IMAX]",2012,5.0
4,101525,"Place Beyond the Pines, The","[Crime, Drama]",2012,3.0
5,102903,Now You See Me,"[Crime, Mystery, Thriller]",2013,3.5


We're going to start by learning the input's preferences, so let's get the subset of movies that the input has watched from the Dataframe containing genres defined with binary values.

In [68]:
#Filtering out the movies from the input
userMovies = moviesWithGenres_df[moviesWithGenres_df['movieId'].isin(inputMovies['movieId'].tolist())]
userMovies

,movieId,title,genres,year,Adventure,Animation,Children,Comedy,Fantasy,Romance,...,Horror,Mystery,Sci-Fi,IMAX,Documentary,War,Musical,Western,Film-Noir,(no genres listed)
7962,8644,"I, Robot","[Action, Adventure, Sci-Fi, Thriller]",2004,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
11624,50872,Ratatouille,"[Animation, Children, Drama]",2007,0.0,1.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
12646,59315,Iron Man,"[Action, Adventure, Sci-Fi]",2008,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
18355,91529,"Dark Knight Rises, The","[Action, Adventure, Crime, IMAX]",2012,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
20807,101525,"Place Beyond the Pines, The","[Crime, Drama]",2012,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
21167,102903,Now You See Me,"[Crime, Mystery, Thriller]",2013,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0



We'll only need the actual genre table, so let's clean this up a bit by resetting the index and dropping the movieId, title, genres and year columns.

In [69]:
#Resetting the index to avoid future issues
userMovies = userMovies.reset_index(drop=True)
#Dropping unnecessary issues due to save memory and to avoid issues
userGenreTable = userMovies.drop('movieId', 1).drop('title', 1).drop('genres', 1).drop('year', 1)
userGenreTable

,Adventure,Animation,Children,Comedy,Fantasy,Romance,Drama,Action,Crime,Thriller,Horror,Mystery,Sci-Fi,IMAX,Documentary,War,Musical,Western,Film-Noir,(no genres listed)
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Now we're ready to start learning the input's preferences!

To do this, we're going to turn each genre into weights. We can do this by using the input's reviews and multiplying them into the input's genre table and then summing up the resulting table by column. This operation is actually a dot product between a matrix and a vector, so we can simply accomplish by calling Pandas's "dot" function.

In [70]:
#Dot produt to get weights
userProfile = userGenreTable.transpose().dot(inputMovies['rating'])
#The user profile
userProfile

Adventure             14.5
Animation              4.0
Children               4.0
Comedy                 0.0
Fantasy                0.0
Romance                0.0
Drama                  7.0
Action                14.5
Crime                 11.5
Thriller               8.5
Horror                 0.0
Mystery                3.5
Sci-Fi                 9.5
IMAX                   5.0
Documentary            0.0
War                    0.0
Musical                0.0
Western                0.0
Film-Noir              0.0
(no genres listed)     0.0
dtype: float64

Now, we have the weights for every of the user's preferences. This is known as the User Profile. Using this, we can recommend movies that satisfy the user's preferences.

In [71]:
#Now let's get the genres of every movie in our original dataframe
genreTable = moviesWithGenres_df.set_index(moviesWithGenres_df['movieId'])
#And drop the unnecessary information
genreTable = genreTable.drop('movieId', 1).drop('title', 1).drop('genres', 1).drop('year', 1)
genreTable.head()

,Adventure,Animation,Children,Comedy,Fantasy,Romance,Drama,Action,Crime,Thriller,Horror,Mystery,Sci-Fi,IMAX,Documentary,War,Musical,Western,Film-Noir,(no genres listed)
movieId,,,,,,,,,,,,,,,,,,,,
1,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


With the input's profile and the complete list of movies and their genres in hand, we're going to take the weighted average of every movie based on the input profile and recommend the top twenty movies that most satisfy it.

In [72]:
#Multiply the genres by the weights and then take the weighted average
recommendationTable_df = ((genreTable*userProfile).sum(axis=1))/(userProfile.sum())
recommendationTable_df.head()

movieId
1    0.274390
2    0.225610
3    0.000000
4    0.085366
5    0.000000
dtype: float64

In [73]:
#Sort our recommendations in descending order
recommendationTable_df = recommendationTable_df.sort_values(ascending=False)
#Just a peek at the values
recommendationTable_df.head()

movieId
115479    0.79878
59844     0.72561
79132     0.72561
81132     0.72561
67070     0.72561
dtype: float64

In [74]:
type(recommendationTable_df)

pandas.core.series.Series

In [75]:
recommended_movies=recommendationTable_df.to_frame('recommendation')
recommended_movies.head()

,recommendation
movieId,
115479,0.79878
59844,0.72561
79132,0.72561
81132,0.72561
67070,0.72561


In [76]:
recommended_movies=recommended_movies.merge(how='left', on='movieId',right=movies_df)
recommended_movies.head(50)

,movieId,recommendation,title,genres,year
0,115479,0.798780,"Whip Hand, The","[Action, Adventure, Crime, Drama, Sci-Fi, Thri...",1951
1,59844,0.725610,"Honor Among Thieves (Adieu l'ami) (Farewell, F...","[Action, Adventure, Crime, Drama, Mystery, Thr...",1968
2,79132,0.725610,Inception,"[Action, Crime, Drama, Mystery, Sci-Fi, Thrill...",2010
3,81132,0.725610,Rubber,"[Action, Adventure, Comedy, Crime, Drama, Film...",2010
4,67070,0.725610,Army of One (Joshua Tree),"[Action, Adventure, Crime, Drama, Mystery, Thr...",1993
5,26701,0.713415,Patlabor: The Movie (Kidô keisatsu patorebâ: T...,"[Action, Animation, Crime, Drama, Film-Noir, M...",1989
6,91054,0.713415,Batman,"[Action, Adventure, Crime, Sci-Fi, Thriller]",1943
7,2625,0.713415,Black Mask (Hak hap),"[Action, Adventure, Crime, Sci-Fi, Thriller]",1996
8,91056,0.695122,Batman and Robin,"[Action, Adventure, Crime, Drama, Sci-Fi]",1949
9,26504,0.689024,Cloak & Dagger,"[Action, Adventure, Children, Crime, Mystery, ...",1984


In [78]:
recommended_movies.to_csv('girlfriend_movies.csv')